In [ ]:
# -*- coding: utf-8 -*-
"""test4i6.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/17BBGfZBN8F57KyoodixZpv1iKiAs-6NT
"""

#test2, with pressure loads
# Cantilever Beam under uniform pressure load
# C:\Users\shivm\OneDrive - Imperial College London\ME4\FYP\Abaqus python scripts

#----------------------------------------------------------------------------------------------------------------------

from abaqus import *
from abaqusConstants import *
import regionToolset


import numpy as np
from itertools import chain
import math


session.viewports['Viewport: 1'].setValues(displayedObject=None)

#----------------------------------------------------------------------------------------------------------------------
# # create the model
mdb.models.changeKey(fromName='Model-1', toName='Cantilever Beam')
beamModel = mdb.models['Cantilever Beam']




length = 5
for j in range(0,7):
  # create the part

  import sketch
  import part

  #a) Sketch the beam cross section using rectangle tool
  beamProfileSketch = beamModel.ConstrainedSketch(name='Beam CS Profile',
                                              sheetSize=5)
  beamProfileSketch.rectangle(point1=(0.1+0.1*j,0.1), point2=(-0.1-0.1*j,-0.1))

  # b) Create a 3d deformable part named "Beam" by extruding the sketch
  beamPart = beamModel.Part(name = 'Beam', dimensionality=THREE_D,
                        type = DEFORMABLE_BODY)
  beamPart.BaseSolidExtrude(sketch=beamProfileSketch, depth=length)
  # #----------------------------------------------------------------------------------------------------------------------
  # create material

  import material

  # Create material AISI 1005 steel by assigning mass density, youngs
  # modulus and poissons ratio
  beamMaterial = beamModel.Material(name='AISI 1005 Steel')
  beamMaterial.Density(table=((7872,),))
  beamMaterial.Elastic(table=((200E9, 0.29),))

  #----------------------------------------------------------------------------------------------------------------------
  # create solid section and assign beam to it

  import section

  # create a section to assign to the beam
  beamSection = beamModel.HomogeneousSolidSection(name='Beam Section', material='AISI 1005 Steel')

  # assign the beam to this section
  beam_region = (beamPart.cells,)
  beamPart.SectionAssignment(region=beam_region, sectionName='Beam Section')

  #----------------------------------------------------------------------------------------------------------------------
  # create the assembly

  import assembly

  # create part instance
  beamAssembly = beamModel.rootAssembly
  beamInstance = beamAssembly.Instance(name='Beam Instance', part=beamPart, dependent=ON)

  #----------------------------------------------------------------------------------------------------------------------
  # create the step

  import step

  # create a static general step
  beamModel.StaticStep(name='Apply Load', previous='Initial', description='Load is applied during this step')

  #----------------------------------------------------------------------------------------------------------------------
  # create field output request

  # change the name of field output request 'F-Output-1' to 'Selected Field Outputs'
  beamModel.fieldOutputRequests.changeKey(fromName='F-Output-1', toName='Selected Field Outputs')

  # since F-Output-1 is applied at the 'Apply Load' step by default, 'Selected Field Outputs' will be too
  # choose which results to output
  beamModel.fieldOutputRequests['Selected Field Outputs'].setValues(variables=('S', 'E', 'PEMAG', 'U', 'RF', 'CF'))

  #----------------------------------------------------------------------------------------------------------------------
  # create the history output request (I don't think I used this section,  just copied it)

  # we try a slightly different method from that used in the field output request
  # create a new history output request called 'Default History Outputs' and assign both the step and the variables
  beamModel.HistoryOutputRequest(name='Default History Outputs', createStepName='Apply Load', variables=PRESELECT)

  # now delete the original history output request 'H-Output-1'
  del beamModel.historyOutputRequests['H-Output-1']

  #----------------------------------------------------------------------------------------------------------------------
  # # apply pressure load
  beamAssembly.Surface(name='Surf-2', side1Faces=
      beamInstance.faces.getSequenceFromMask(
      ('[#8 ]', ), ))
  beamAssembly.Surface(name='Surf-3', side1Faces=
      beamInstance.faces.getSequenceFromMask(
      ('[#8 ]', ), ))
  beamModel.Pressure(amplitude=UNSET, createStepName=
      'Apply Load', distributionType=UNIFORM, field='', magnitude=5.0, name=
      'Load-1', region=
      beamAssembly.surfaces['Surf-3'])



  #----------------------------------------------------------------------------------------------------------------------
  # apply encastre (fixed) boundary condition on one end to make it a cantilever
  beamModel.EncastreBC(createStepName='Initial', name=
      'Encaster one end', region=regionToolset.Region(
      vertices=beamInstance.vertices[0:0]))
  beamAssembly.Set(faces=
      beamInstance.faces.getSequenceFromMask(
      ('[#10 ]', ), ), name='Set-2')
  beamModel.boundaryConditions['Encaster one end'].setValues(
      region=beamAssembly.sets['Set-2'])


  mdb.saveAs(pathName='C:\Abaqus_sims\Tests\Test6cae')
  #----------------------------------------------------------------------------------------------------------------------
  # create the mesh

  import mesh

  # first we need to locate and select a point inside the solid
  # we place a point somewhere inside it based on out knowledge of the geometry
  beam_inside_xcoord=0
  beam_inside_ycoord=0
  beam_inside_zcoord=2.5

  # select element type. CAX4R is a 4 node axisymmetric element
  elemType1 = mesh.ElemType(elemCode=C3D20R, elemLibrary=STANDARD, kinematicSplit=AVERAGE_STRAIN, secondOrderAccuracy=OFF,
                            hourglassControl=DEFAULT, distortionControl=DEFAULT)

  # select the region to mesh
  beamCells=beamPart.cells
  selectedBeamCells=beamCells.findAt((beam_inside_xcoord,beam_inside_ycoord,beam_inside_zcoord),)

  beamMeshRegion=(selectedBeamCells,)
  beamPart.setElementType(regions=beamMeshRegion, elemTypes=(elemType1,))

  # select mesh refinement and mesh the part
  beamPart.seedPart(size=0.05, deviationFactor=0.1)
  beamPart.generateMesh()

  # create and run the job

  import job

  # create the job
  # the job number is incremented with every loop
  mdb.Job(name='CantileverBeamJob', model='Cantilever Beam', type=ANALYSIS, explicitPrecision=SINGLE,
          nodalOutputPrecision=SINGLE, description='Job simulates a loaded cantilever beam',
          parallelizationMethodExplicit=DOMAIN, multiprocessingMode=DEFAULT, numDomains=1, userSubroutine='',
          numCpus=1, memory=50, memoryUnits=PERCENTAGE, scratch='', echoPrint=OFF, modelPrint=OFF,
          contactPrint=OFF, historyPrint=OFF)

  # run the job
  mdb.jobs['CantileverBeamJob'].submit(consistencyChecking=OFF)

  # Do not return control till job is finished running
  mdb.jobs['CantileverBeamJob'].waitForCompletion()

  # end of run job
  # ----------------------------------------------------------------------------------------------------------------------
  # post processing

  import visualization

  # the output database is named
  beam_viewport = session.Viewport(name='Beam Results Viewport')
  beam_Odb_Path = 'CantileverBeamJob.odb'
  an_odb_object = session.openOdb(name=beam_Odb_Path)
  beam_viewport.setValues(displayedObject=an_odb_object)
  beam_viewport.odbDisplay.display.setValues(plotState=(DEFORMED, ))

  session.viewports['Viewport: 1'].setValues(displayedObject=an_odb_object)
  # set report name and format for each job
  report_name_and_path = 'Report'+str(j)+'.csv'
  #report_name_and_path = 'Report'+'.csv'

  # writes the report with the variables you choose. I've chosen sigma_33 at every integration point
  session.fieldReportOptions.setValues(reportFormat=COMMA_SEPARATED_VALUES)
  session.writeFieldReport(fileName=report_name_and_path, append=OFF,
      sortItem='Element Label', odb=an_odb_object, step=0, frame=1,
      outputPosition=INTEGRATION_POINT, variable=(('S', INTEGRATION_POINT, ((
      COMPONENT, 'S33'), )), ))
  
  #we want the stresses at these points(0, l/4, l/2, 3/4l and l)
  
  length += j